In [1]:
# 조회? 하는 최근접 취약시설의 수
n_neighbors = 5

In [2]:
# !pip install --break-system-packages scikit-learn
# !pip install --break-system-packages math

# !pip install scikit-learn
# !pip install math
# 필요한 라이브러리 임포트

In [3]:
import numpy as np
from sklearn.neighbors import KDTree
from math import radians, sin, cos, sqrt, atan2
import pandas as pd

In [4]:
# 데이터 불러오기
road_url = './make_file/열선_위경도값(중앙값_열선_경사각_포함).csv'
facility_url = '../서울시 취약시설 데이터 수집/make_file/(최종)_서울시_모든_시설의_위치.csv'

In [5]:
facility_data = pd.read_csv(facility_url)
road_data = pd.read_csv(road_url, encoding = "EUC_KR")

In [6]:
facility_data.columns

Index(['Unnamed: 0', '시설_구분', '시설_이름', '시설_위치', '위도', '경도'], dtype='object')

In [7]:
road_data.columns

Index(['Unnamed: 0', '시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이', '시작점_고도',
       '종료점_고도', '고도의_차이', '경사각', '중점_위도', '중점_경도'],
      dtype='object')

In [8]:

# 도로 중앙값 데이터 (위경도)
road_lat_list = road_data['종점_위도'].to_list()
road_lon_list = road_data['종점_경도'].to_list()

# 취약시설 데이터 (시설명, 위경도) 
facility_lat_list = facility_data["위도"].to_list()
facility_lon_list = facility_data["경도"].to_list()


In [9]:
print(f"road_lat_list 길이 : {len(road_lat_list)}")
print(f"road_lon_list 길이 : {len(road_lon_list)}")
 
print(f"facility_lat_list 길이 : {len(facility_lat_list)}")
print(f"facility_lon_list 길이 : {len(facility_lon_list)}")

road_lat_list 길이 : 485
road_lon_list 길이 : 485
facility_lat_list 길이 : 11908
facility_lon_list 길이 : 11908


In [10]:
# 거리를 구하기
def haversine(lat1, lon1, lat2, lon2): 
    R = 6371  # 지구 반경 (km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    result = R * c * 1000 
    return result # 미터 단위로 반환

In [11]:
def find_nearest_facilities_with_haversine(road_lats, road_lons, facility_lats, facility_lons):
    global n_neighbors
    # KDTree로 시설 좌표 인덱스를 트리 구조로 저장
    facilities = np.array(list(zip(facility_lats, facility_lons)))
    tree = KDTree(facilities)

    # 각 도로에 대해 최근접 n개의 시설 찾기
    distance_matrix = []
    avg_distances = []
    
    for r_lat, r_lon in zip(road_lats, road_lons):
        road_coords = np.array([[r_lat, r_lon]])
        
        # 최근접 n개 시설의 인덱스 및 거리 계산
        dist, indices = tree.query(road_coords, k=n_neighbors)
        
        # 거리 계산하여 2차원 배열에 저장
        distances = []
        for i in range(n_neighbors):
            dist_val = haversine(r_lat, r_lon, facility_lats[indices[0][i]], facility_lons[indices[0][i]])  # Haversine 거리 계산
            distances.append(dist_val)
        
        distance_matrix.append(distances)
        
        # 평균 거리 계산하여 저장
        avg_distance = np.mean(distances)
        avg_distances.append(avg_distance)
    
    return np.array(distance_matrix), np.array(avg_distances)

In [12]:
# 거리 계산
distance_matrix, avg_distances = find_nearest_facilities_with_haversine(road_lat_list, road_lon_list, facility_lat_list, facility_lon_list)

In [13]:
print(f"수집한 도로_취약시설 거리의 수 : {len(distance_matrix)}")
print(f"수집한 도로_취약시설 거리의 평균 수 : {len(avg_distances)}")

수집한 도로_취약시설 거리의 수 : 485
수집한 도로_취약시설 거리의 평균 수 : 485


In [14]:
print(distance_matrix[1])

[ 140.32634861  585.94339803  586.37106416  864.16474458 1034.94296514]


In [15]:
distance_matrix = distance_matrix.tolist()

In [16]:
new_distance_matrix = [" ,".join(map(str, row)) for row in distance_matrix]
new_distance_matrix[1]

'140.32634860509523 ,585.9433980328229 ,586.3710641562539 ,864.1647445754376 ,1034.9429651352716'

In [17]:
road_data['최근접_시설들_거리'] = new_distance_matrix
road_data['최근접_시설의_평균거리'] = avg_distances

In [18]:
print(road_data['최근접_시설의_평균거리'].mean())

183.18153034162052


In [19]:
# road_data['최근접_시설들_거리'] 

In [20]:
try :
    road_data = road_data.drop(columns='Unnamed: 0')
    print(f"Unnamed: 0 삭제함.")
except KeyError as e:
    print(e)

Unnamed: 0 삭제함.


In [21]:
road_data.columns


Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이', '시작점_고도', '종료점_고도', '고도의_차이',
       '경사각', '중점_위도', '중점_경도', '최근접_시설들_거리', '최근접_시설의_평균거리'],
      dtype='object')

In [22]:
save_file_url = './make_file/(최종)_열선_취약시설_거리.csv'
save_file_encoding = 'EUC-KR'
# save_file_encoding = 'UTF-8'

In [23]:
try :
    road_data.to_csv(save_file_url, encoding=save_file_encoding, index=False)
    print(f"{save_file_url} 파일 저장을 성공했습니다.")
except OSError as e:
    print(e)

./make_file/(최종)_열선_취약시설_거리.csv 파일 저장을 성공했습니다.


In [24]:
road_data

,시점_위도,시점_경도,종점_위도,종점_경도,길이,시작점_고도,종료점_고도,고도의_차이,경사각,중점_위도,중점_경도,최근접_시설들_거리,최근접_시설의_평균거리
0,37.589926,126.993697,37.591160,126.992929,160,62.080006,93.832680,31.752674,11.724227,37.590543,126.993313,"152.84881446864983 ,181.9200545209481 ,262.128...",219.445340
1,37.611574,126.978609,37.615247,126.976917,150,149.520370,187.892639,38.372269,5.044559,37.613411,126.977763,"140.32634860509523 ,585.9433980328229 ,586.371...",642.349704
2,37.610064,126.975395,37.612667,126.975238,210,118.210579,158.207809,39.997231,7.859327,37.611366,126.975317,"229.34052126931942 ,267.88608827310566 ,266.87...",420.360320
3,37.606240,126.957833,37.604560,126.957061,190,92.292526,127.500053,35.207527,10.043141,37.605400,126.957447,"145.39902591309126 ,174.8559555300938 ,174.855...",207.818466
4,37.583442,126.985409,37.583186,126.988885,355,48.981506,42.394905,6.586601,1.226563,37.583314,126.987147,"141.34252205592068 ,141.34252205592068 ,196.29...",250.589899
...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,37.474572,126.963120,37.474250,126.961186,20,58.153191,54.468281,3.684910,1.210493,37.474411,126.962153,"115.68447472338539 ,137.37240451866106 ,166.25...",191.188786
481,37.486210,126.957150,37.486370,126.959313,120,49.216015,59.009739,9.793724,2.924716,37.486290,126.958232,"38.99147119223848 ,76.31678766060915 ,67.29862...",70.341129
482,37.474302,126.976014,37.473301,126.973641,85,28.454460,49.099812,20.645351,4.974049,37.473802,126.974827,"62.50475809003016 ,158.56044484349556 ,203.242...",191.033581
483,37.469265,126.965549,37.469517,126.966069,250,88.539551,91.336472,2.796921,2.978165,37.469391,126.965809,"184.26448747692467 ,228.4391830090743 ,317.761...",279.631097
